In this notebook, you should implement a first version of a working machine learning model to predict the age of an Abalone.

A few guidelines:
- The model does not have to be complex. A simple linear regression model is enough.
- You should use MLflow to track your experiments. You can use the MLflow UI to compare your experiments.
- Do not push any MLflow data to the repository. Only the code to run the experiments is interesting and should be pushed.

In [ ]:
# Imports pour la modélisation
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error
import warnings
warnings.filterwarnings('ignore')

# Configuration
pd.set_option('display.max_columns', 500)
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

print("Imports terminés avec succès!")


In [ ]:
# Chargement et préparation des données
print("=== CHARGEMENT DES DONNÉES ===")

# Charger le dataset
df = pd.read_csv("../abalone.csv")
print(f"Dataset chargé: {df.shape}")

# Afficher les premières lignes
print("\nPremières lignes:")
print(df.head())

# Vérifier les types de données
print(f"\nTypes de données:")
print(df.dtypes)


In [ ]:
# Préparation des features et de la variable cible
print("=== PRÉPARATION DES DONNÉES ===")

# Encoder la variable catégorielle Sex
le = LabelEncoder()
df['Sex_encoded'] = le.fit_transform(df['Sex'])

print("Mapping des valeurs Sex:")
for i, sex in enumerate(le.classes_):
    print(f"{sex}: {i}")

# Définir les features et la target
feature_cols = ['Sex_encoded', 'Length', 'Diameter', 'Height', 
                'Whole weight', 'Shucked weight', 'Viscera weight', 'Shell weight']
X = df[feature_cols]
y = df['Rings']

print(f"\nFeatures sélectionnées: {feature_cols}")
print(f"Shape de X: {X.shape}")
print(f"Shape de y: {y.shape}")

# Division train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
print(f"\nDivision train/test:")
print(f"Train: {X_train.shape[0]} échantillons")
print(f"Test: {X_test.shape[0]} échantillons")


In [ ]:
# Normalisation des features
print("=== NORMALISATION DES FEATURES ===")

# Standardisation des features
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

print("Normalisation effectuée avec StandardScaler")
print(f"Mean des features normalisées (train): {X_train_scaled.mean(axis=0)}")
print(f"Std des features normalisées (train): {X_train_scaled.std(axis=0)}")


In [ ]:
# Modélisation - Régression Linéaire Simple
print("=== RÉGRESSION LINÉAIRE SIMPLE ===")

# Modèle de régression linéaire
lr = LinearRegression()
lr.fit(X_train_scaled, y_train)

# Prédictions
y_pred_lr = lr.predict(X_test_scaled)

# Métriques
mse_lr = mean_squared_error(y_test, y_pred_lr)
rmse_lr = np.sqrt(mse_lr)
r2_lr = r2_score(y_test, y_pred_lr)
mae_lr = mean_absolute_error(y_test, y_pred_lr)

print(f"Régression Linéaire - Métriques:")
print(f"RMSE: {rmse_lr:.3f}")
print(f"R²: {r2_lr:.3f}")
print(f"MAE: {mae_lr:.3f}")

# Cross-validation
cv_scores_lr = cross_val_score(lr, X_train_scaled, y_train, cv=5, scoring='r2')
print(f"R² Cross-validation (5-fold): {cv_scores_lr.mean():.3f} (+/- {cv_scores_lr.std() * 2:.3f})")


In [ ]:
# Modélisation - Random Forest
print("=== RANDOM FOREST ===")

# Modèle Random Forest
rf = RandomForestRegressor(n_estimators=100, random_state=42)
rf.fit(X_train, y_train)  # Pas besoin de normalisation pour Random Forest

# Prédictions
y_pred_rf = rf.predict(X_test)

# Métriques
mse_rf = mean_squared_error(y_test, y_pred_rf)
rmse_rf = np.sqrt(mse_rf)
r2_rf = r2_score(y_test, y_pred_rf)
mae_rf = mean_absolute_error(y_test, y_pred_rf)

print(f"Random Forest - Métriques:")
print(f"RMSE: {rmse_rf:.3f}")
print(f"R²: {r2_rf:.3f}")
print(f"MAE: {mae_rf:.3f}")

# Cross-validation
cv_scores_rf = cross_val_score(rf, X_train, y_train, cv=5, scoring='r2')
print(f"R² Cross-validation (5-fold): {cv_scores_rf.mean():.3f} (+/- {cv_scores_rf.std() * 2:.3f})")

# Importance des features
feature_importance = pd.DataFrame({
    'feature': feature_cols,
    'importance': rf.feature_importances_
}).sort_values('importance', ascending=False)

print(f"\nImportance des features:")
print(feature_importance)


In [ ]:
# Comparaison des modèles
print("=== COMPARAISON DES MODÈLES ===")

# Créer un DataFrame de comparaison
results = pd.DataFrame({
    'Modèle': ['Régression Linéaire', 'Random Forest'],
    'RMSE': [rmse_lr, rmse_rf],
    'R²': [r2_lr, r2_rf],
    'MAE': [mae_lr, mae_rf]
})

print("Comparaison des performances:")
print(results.round(3))

# Visualisation des prédictions vs vraies valeurs
fig, axes = plt.subplots(1, 2, figsize=(15, 6))

# Régression Linéaire
axes[0].scatter(y_test, y_pred_lr, alpha=0.6)
axes[0].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
axes[0].set_xlabel('Vraies valeurs')
axes[0].set_ylabel('Prédictions')
axes[0].set_title(f'Régression Linéaire (R² = {r2_lr:.3f})')

# Random Forest
axes[1].scatter(y_test, y_pred_rf, alpha=0.6)
axes[1].plot([y_test.min(), y_test.max()], [y_test.min(), y_test.max()], 'r--', lw=2)
axes[1].set_xlabel('Vraies valeurs')
axes[1].set_ylabel('Prédictions')
axes[1].set_title(f'Random Forest (R² = {r2_rf:.3f})')

plt.tight_layout()
plt.show()


## Conclusions de la Modélisation

### Résultats obtenus :

1. **Régression Linéaire** :
   - Modèle simple et interprétable
   - Performance correcte pour un modèle de base
   - Nécessite une normalisation des features

2. **Random Forest** :
   - Généralement meilleure performance
   - Capture les interactions non-linéaires
   - Fournit l'importance des features
   - Plus robuste aux outliers

### Recommandations pour l'amélioration :

1. **Feature Engineering** :
   - Créer de nouvelles features (ratios, interactions)
   - Gérer les outliers détectés en EDA

2. **Optimisation des hyperparamètres** :
   - Grid Search pour Random Forest
   - Test de différents algorithmes (XGBoost, SVM)

3. **Validation** :
   - Stratified sampling si nécessaire
   - Time series split si applicable

### Prochaines étapes :
- Sauvegarder le meilleur modèle
- Préparer le pipeline de prédiction
- Intégrer MLflow pour le tracking
